In [10]:
import pickle

fin = open("datasets/test_expr_values.pkl", "rb")
value_sets = pickle.load(fin)
expr_values = pickle.load(fin)
fin.close()

In [13]:
import itertools
from tqdm import tqdm
from collections import defaultdict

# # Example expr_values dictionary
# # Replace this with your actual data
# expr_values = {
#     # SymPy expressions as keys (using strings for simplicity)
#     'expr1': [1.0, 2.0, 3.0, 4.0, 5.0, 16.0, 17.0, 18.0, 9.0, 10.0],
#     'expr2': [0.0, 2.0, 3.0, 4.0, 5.0, 16.0, 17.0, 18.0, 19.0, 10.0],
#     'expr3': [1.0, 2.0, 3.0, 4.0, 5.0, 16.0, 17.0, 18.0, 19.0, 20.0],
#     'expr4': [11.0, 12.0, 13.0, 14.0, 15.0, 6.0, 7.0, 8.0, 9.0, 10.0],
#     'expr5': [1.0, 2.0, 3.0, 4.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0],
#     # Add more expressions as needed
# }


def find_overlapping_pairs(expr_values, value_size, min_overlap):
    # Step 1: Create a mapping from signatures to expressions
    signature_map = defaultdict(set)

    # Function to generate signatures and update the signature_map
    def generate_signatures(expr_key, values_list):
        positions = list(range(len(values_list)))  # Positions from 0 to 9
        # Generate subsets of positions of sizes 8, 9, and 10
        for r in range(min_overlap, value_size+1):
            position_subsets = itertools.combinations(positions, r)
            for pos_subset in position_subsets:
                # Extract values at the selected positions
                value_subset = tuple((pos, values_list[pos]) for pos in pos_subset)
                # Use the value_subset as a signature
                signature_map[value_subset].add(expr_key)

    # Populate the signature_map
    print("Generating signatures of expressions")
    for expr_key, values_list in tqdm(expr_values.items()):
        generate_signatures(expr_key, values_list)

    # Step 2: Find all pairs of expressions sharing at least 8 common values at the same positions
    result_pairs = set()

    print("Checking signatures")
    for signature, expr_set in tqdm(signature_map.items()):
        if len(expr_set) > 1:
            # Generate all unique pairs of expressions
            expr_list = list(expr_set)
            for i in range(len(expr_list)):
                for j in range(i + 1, len(expr_list)):
                    pair = tuple([expr_list[i], expr_list[j]])
                    result_pairs.add(pair)

    # Step 3: Output the pairs
    #print("Expressions sharing at least 8 common values at the same positions:")
    #for pair in sorted(result_pairs):
    #    print(f"{pair[0]} and {pair[1]}")
        
    return result_pairs

equiv_pairs = find_overlapping_pairs(expr_values, 10, 8)

Generating signatures of expressions


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40424/40424 [00:13<00:00, 2924.82it/s]


Checking signatures


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2198249/2198249 [00:00<00:00, 9411891.87it/s]


In [7]:
import sympy as sp
from sympy import symbols, simplify, sympify
from collections import defaultdict

# Define the symbol 'a' used in the expressions
a = symbols('a')

# List of equivalent expression pairs
pairs = [
    ('0', 'a * 0'),
    ('0', 'a - a'),
    ('0', '0 * 0'),
    ('0', '0 - 0'),
    ('a', 'a + 0'),
    ('a', 'a - 0'),
    ('0 * a', '0 - 0'),
    ('0 * a', '1 * 0'),
    ('a * 1', 'a + 0'),
    ('0 * a', '0 + 0'),
    ('0 * 0', 'a * 0'),
    ('(1 * 0) + a', 'a'),
    ('(0 + 0) + 1', '1'),
    ('(1 * 1) * a', 'a'),
    ('(1 * a) + 0', 'a'),
    ('(0 * 0) + a', 'a'),
    ('(a - 0) - 0', 'a'),
    ('1', '1 + (0 * 1)'),
    ('((a - 0) - a) + b', 'b'),
]


def create_shortening_mappings(pairs):
    mapping = defaultdict(set)
    for e1, e2 in pairs:
        if len(e1) > len(e2):
            mapping[e1].add(e2)
        if len(e1) < len(e2):
            mapping[e2].add(e1)
    return mapping
            
def find_parentheses_substrings(s):
    stack = []
    substrings = []
    for i, c in enumerate(s):
        if c == '(':
            stack.append(i)
        elif c == ')':
            if stack:
                j = stack.pop()
                substrings.append(s[j:i+1])
            else:
                # Handle unmatched closing parenthesis if necessary
                pass
    return substrings

def shorten_expr(expr, mapping):
    output = []
    #if expr in mapping:
    #    output.extend(mapping[expr])
    parentheses_exprs = find_parentheses_substrings(expr)
    for parentheses_expr in parentheses_exprs:
        sub_expr = parentheses_expr[1:-1]
        if sub_expr in mapping:
            for short_expr in mapping[sub_expr]:
                if len(short_expr) > 2:
                    new_expr = expr.replace(parentheses_expr, '(' + short_expr + ')')
                else:
                    new_expr = expr.replace(parentheses_expr, short_expr)
                output.append(new_expr)
    return output

def repeated_shorten_expr(expr, mapping):
    attempts = 5
    while attempts > 0:
        shorted_exprs = shorten_expr(expr, mapping)
        if len(shorted_exprs) == 0:
            return expr
        else:
            expr = min(shorted_exprs, key=len)
            attempts -= 1
    return expr

mapping = create_shortening_mappings(pairs)
for e1, e2 in pairs:
    print("\nEXPR:", e1)
    shorted_exprs = repeated_shorten_expr(e1, mapping)
    print(shorted_exprs)
    



EXPR: 0
0

EXPR: 0
0

EXPR: 0
0

EXPR: 0
0

EXPR: a
a

EXPR: a
a

EXPR: 0 * a
0 * a

EXPR: 0 * a
0 * a

EXPR: a * 1
a * 1

EXPR: 0 * a
0 * a

EXPR: 0 * 0
0 * 0

EXPR: (1 * 0) + a
(1 * 0) + a

EXPR: (0 + 0) + 1
(0 + 0) + 1

EXPR: (1 * 1) * a
(1 * 1) * a

EXPR: (1 * a) + 0
(1 * a) + 0

EXPR: (0 * 0) + a
0 + a

EXPR: (a - 0) - 0
a - 0

EXPR: 1
1

EXPR: ((a - 0) - a) + b
0 + b


In [4]:
def find_parentheses_substrings(s):
    stack = []
    substrings = []
    for i, c in enumerate(s):
        if c == '(':
            stack.append(i)
        elif c == ')':
            if stack:
                j = stack.pop()
                substrings.append(s[j:i+1])
            else:
                # Handle unmatched closing parenthesis if necessary
                pass
    return substrings

expression = '((a - 1) - (b - c)) - 0'
result = find_parentheses_substrings(expression)
print(result)

['(a - 1)', '(b - c)', '((a - 1) - (b - c))']
